Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

https://stackoverflow.com/questions/38286717/tensorflow-regularization-with-l2-loss-how-to-apply-to-all-weights-not-jus

In [5]:
def create_nn(dataset, weights_hidden, biases_hidden, weights_out, biases_out):
    # Original layer
    logits = tf.add(tf.matmul(tf_train_dataset, weights_hidden), biases_hidden)
    # Hidden Relu layer
    logits = tf.nn.relu(logits)
    # Output: Connect hidden layer to a node for each class
    logits = tf.add(tf.matmul(logits, weights_out), biases_out)
    return logits
    
    
# Create model
batch_size = 128
hidden_layer_size = 1024
beta = 1e-3

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden = tf.Variable(
        #tf.truncated_normal([image_size * image_size, num_labels]))
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    #biases = tf.Variable(tf.zeros([num_labels]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

    weights_out = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_out = tf.Variable(tf.zeros([num_labels]))
    
    
    # Training computation.
    #logits = tf.matmul(tf_train_dataset, weights_out) + biases_out
    logits = create_nn(tf_train_dataset, weights_hidden, biases_hidden, weights_out, biases_out)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_out))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights_out) + biases_out)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights_out) + biases_out)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)


In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 680.027222
Minibatch accuracy: 6.2%
Validation accuracy: 32.9%
Minibatch loss at step 500: 200.277863
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 115.910881
Minibatch accuracy: 75.0%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 68.614128
Minibatch accuracy: 83.6%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 41.258678
Minibatch accuracy: 89.1%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 25.068829
Minibatch accuracy: 90.6%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.448822
Minibatch accuracy: 88.3%
Validation accuracy: 86.7%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
def create_nn(dataset, weights_hidden, biases_hidden, weights_out, biases_out):
    # Original layer
    logits = tf.add(tf.matmul(tf_train_dataset, weights_hidden), biases_hidden)
    # Hidden Relu layer
    logits = tf.nn.relu(logits)
    # Output: Connect hidden layer to a node for each class
    logits = tf.add(tf.matmul(logits, weights_out), biases_out)
    return logits
    
    
# Create model
batch_size = 128
hidden_layer_size = 1024
beta = 1e-3

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden = tf.Variable(
        #tf.truncated_normal([image_size * image_size, num_labels]))
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    #biases = tf.Variable(tf.zeros([num_labels]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

    weights_out = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_out = tf.Variable(tf.zeros([num_labels]))
    
    
    # Training computation.
    #logits = tf.matmul(tf_train_dataset, weights_out) + biases_out
    logits = create_nn(tf_train_dataset, weights_hidden, biases_hidden, weights_out, biases_out)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    #loss += beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_out))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights_out) + biases_out)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights_out) + biases_out)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)


In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (3*128 - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 337.697693
Minibatch accuracy: 7.8%
Validation accuracy: 37.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Test accuracy: 77.0%


How does it perform with L2 regularization on weights?

In [9]:
def create_nn(dataset, weights_hidden, biases_hidden, weights_out, biases_out):
    # Original layer
    logits = tf.add(tf.matmul(tf_train_dataset, weights_hidden), biases_hidden)
    # Hidden Relu layer
    logits = tf.nn.relu(logits)
    # Output: Connect hidden layer to a node for each class
    logits = tf.add(tf.matmul(logits, weights_out), biases_out)
    return logits
    
    
# Create model
batch_size = 128
hidden_layer_size = 1024
beta = 1e-3

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden = tf.Variable(
        #tf.truncated_normal([image_size * image_size, num_labels]))
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    #biases = tf.Variable(tf.zeros([num_labels]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

    weights_out = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_out = tf.Variable(tf.zeros([num_labels]))
    
    
    # Training computation.
    #logits = tf.matmul(tf_train_dataset, weights_out) + biases_out
    logits = create_nn(tf_train_dataset, weights_hidden, biases_hidden, weights_out, biases_out)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_out))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights_out) + biases_out)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights_out) + biases_out)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)


In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (3*128 - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 637.670044
Minibatch accuracy: 4.7%
Validation accuracy: 26.8%
Minibatch loss at step 500: 190.413055
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 1000: 115.476898
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 1500: 70.031532
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 2000: 42.470997
Minibatch accuracy: 100.0%
Validation accuracy: 73.6%
Minibatch loss at step 2500: 25.756893
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 3000: 15.620900
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Test accuracy: 81.5%


Slightly better

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
def create_nn(dataset, weights_hidden, biases_hidden, weights_out, biases_out):
    # Original layer
    logits = tf.add(tf.matmul(tf_train_dataset, weights_hidden), biases_hidden)
    # Drop Out layer 1
    logits = tf.nn.dropout(logits, 0.5)
    # Hidden Relu layer
    logits = tf.nn.relu(logits)
    # Drop Out layer 2
    logits = tf.nn.dropout(logits, 0.5)
    # Output: Connect hidden layer to a node for each class
    logits = tf.add(tf.matmul(logits, weights_out), biases_out)
    return logits
    
    
# Create model
batch_size = 128
hidden_layer_size = 1024
beta = 1e-3

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden = tf.Variable(
        #tf.truncated_normal([image_size * image_size, num_labels]))
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    #biases = tf.Variable(tf.zeros([num_labels]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

    weights_out = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_out = tf.Variable(tf.zeros([num_labels]))
    
    
    # Training computation.
    #logits = tf.matmul(tf_train_dataset, weights_out) + biases_out
    logits = create_nn(tf_train_dataset, weights_hidden, biases_hidden, weights_out, biases_out)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_out))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights_out) + biases_out)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights_out) + biases_out)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)


num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (3*128 - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 969.488220
Minibatch accuracy: 13.3%
Validation accuracy: 34.4%
Minibatch loss at step 500: 200.872772
Minibatch accuracy: 96.9%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 120.353645
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 73.118279
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 2000: 44.383736
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2500: 26.930599
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 16.340750
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Test accuracy: 84.7%


With both regularizations (l2 and dropout) we got to 84% with only 3 batches!

Now try this with all batches -- who high do we go?

In [12]:
def create_nn(dataset, weights_hidden, biases_hidden, weights_out, biases_out):
    # Original layer
    logits = tf.add(tf.matmul(tf_train_dataset, weights_hidden), biases_hidden)
    # Drop Out layer 1
    #logits = tf.nn.dropout(logits, 0.5)
    # Hidden Relu layer
    logits = tf.nn.relu(logits)
    # Drop Out layer 2
    #logits = tf.nn.dropout(logits, 0.5)
    # Output: Connect hidden layer to a node for each class
    logits = tf.add(tf.matmul(logits, weights_out), biases_out)
    return logits
    
    
# Create model
batch_size = 128
hidden_layer_size = 1024
beta = 1e-3

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden = tf.Variable(
        #tf.truncated_normal([image_size * image_size, num_labels]))
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    #biases = tf.Variable(tf.zeros([num_labels]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

    weights_out = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_out = tf.Variable(tf.zeros([num_labels]))
    
    
    # Training computation.
    #logits = tf.matmul(tf_train_dataset, weights_out) + biases_out
    logits = create_nn(tf_train_dataset, weights_hidden, biases_hidden, weights_out, biases_out)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_out))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights_out) + biases_out)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights_out) + biases_out)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights_out) + biases_out)


num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #offset = (step * batch_size) % (3*128 - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 632.247070
Minibatch accuracy: 11.7%
Validation accuracy: 22.3%
Minibatch loss at step 500: 198.850281
Minibatch accuracy: 80.5%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 115.369583
Minibatch accuracy: 80.5%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 68.523018
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 41.208591
Minibatch accuracy: 91.4%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.116470
Minibatch accuracy: 89.1%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 15.438393
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 3500: 9.562794
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 4000: 5.951413
Minibatch accuracy: 91.4%
Validation accuracy: 87.3%
Minibatch loss at step 4500: 3.790181
Minibatch accuracy: 89.8%
Validation accuracy: 87.4%
Minibatch loss at step 5000: 2.575710
Minibatch accuracy: 87.5%
Validati

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Trying both L2 and dropout to improve score. It didn't work with dropout

In [13]:
# Create model
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 256
num_hidden_nodes3 = 128
beta = 1e-3

def create_nn(dataset, parameters):
    for i in range(len(parameters)):
        if i == 0:
            logits = tf.matmul(dataset, parameters[i][0]) + parameters[i][1]
        else:
            logits = tf.matmul(logits, parameters[i][0]) + parameters[i][1]
        
        # Add a Relu if this isn't the final set of weights. Note: failed to work without a Relu. Why?
        if i < len(parameters)-1:
            # Relu
            logits = tf.nn.relu(logits)

    return logits
    
    

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)

    # Variables. Note: to make this work, I had to add stddev parameter. Why?
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes1],stddev=np.sqrt(2.0 / (image_size * image_size))))
    b1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
    W2 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2],stddev=np.sqrt(2.0 / (num_hidden_nodes1))))
    b2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
    W3 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3],stddev=np.sqrt(2.0 / (num_hidden_nodes2))))
    b3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
    weights_out = tf.Variable(tf.truncated_normal([num_hidden_nodes3, num_labels],stddev=np.sqrt(2.0 / (num_hidden_nodes3))))
    biases_out = tf.Variable(tf.zeros([num_labels]))
    parameters = [(W1, b1), (W2, b2), (W3, b3), (weights_out, biases_out)]
    
    
    # Training computation.
    train_logits = create_nn(tf_train_dataset, parameters)
    test_logits = create_nn(tf_test_dataset, parameters)
    valid_logits = create_nn(tf_valid_dataset, parameters)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits))
    # Add L2 regularization. Note: Makes it worse with deep network for some reason
#     for i in range(len(parameters)-1):
#         loss += beta * tf.nn.l2_loss(parameters[i][0])

    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
#     # Optimizer.
#     optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

num_steps = 18001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #offset = (step * batch_size) % (3*128 - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 2.329853
Minibatch accuracy: 14.1%
Validation accuracy: 33.0%
Minibatch loss at step 500: 0.387847
Minibatch accuracy: 88.3%
Validation accuracy: 86.1%
Minibatch loss at step 1000: 0.486983
Minibatch accuracy: 86.7%
Validation accuracy: 87.1%
Minibatch loss at step 1500: 0.312811
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 2000: 0.258745
Minibatch accuracy: 95.3%
Validation accuracy: 88.9%
Minibatch loss at step 2500: 0.203082
Minibatch accuracy: 94.5%
Validation accuracy: 89.3%
Minibatch loss at step 3000: 0.309711
Minibatch accuracy: 89.8%
Validation accuracy: 89.6%
Minibatch loss at step 3500: 0.302288
Minibatch accuracy: 91.4%
Validation accuracy: 90.2%
Minibatch loss at step 4000: 0.201832
Minibatch accuracy: 94.5%
Validation accuracy: 90.2%
Minibatch loss at step 4500: 0.215523
Minibatch accuracy: 93.0%
Validation accuracy: 90.2%
Minibatch loss at step 5000: 0.261133
Minibatch accuracy: 92.2%
Validation accurac

Example from https://github.com/budmitr/udacity-deep-learning/blob/master/3_regularization.ipynb
They don't use stddev and use dropout and both seem to work

In [16]:
batch_size = 128
hidden_size = 1024
num_steps=10001
reg = 1e-3
dropout = 0.5


graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    b1 = tf.Variable(tf.zeros([hidden_size]))
    W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)

    logits = tf.matmul(tf.nn.dropout(h1, dropout), W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    loss += reg * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))

    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.9)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


    train_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1) * dropout, W2) + b2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1) * dropout, W2) + b2)
    test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,  W1) + b1) * dropout, W2) + b2)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")

    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 791.658447
Minibatch accuracy: 10.2%
Validation accuracy: 28.6%
Minibatch loss at step 500: 218.778015
Minibatch accuracy: 82.0%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 125.295578
Minibatch accuracy: 78.1%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 79.520325
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 50.451607
Minibatch accuracy: 89.1%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 33.924656
Minibatch accuracy: 84.4%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 23.557600
Minibatch accuracy: 85.2%
Validation accuracy: 84.4%
Minibatch loss at step 3500: 16.560410
Minibatch accuracy: 87.5%
Validation accuracy: 85.1%
Minibatch loss at step 4000: 11.956594
Minibatch accuracy: 89.1%
Validation accuracy: 85.2%
Minibatch loss at step 4500: 8.795779
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Minibatch loss at step 5000: 6.664379
Minibatch accuracy: 89.1%
Valida